In [47]:
import tensorflow as tf
import numpy as np
import logging
import tqdm

In [25]:
# import tensorflow as tf
# print (tf.__version__)
# if tf.test.gpu_device_name():
#     print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
# else:
#     print("Please install GPU version of TF")


In [26]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

### 使用层归一layer normalization

In [27]:
def ln(inputs, epsilon=1e-8, scope="ln"):
    """
            使用层归一layer normalization
        tensorflow 在实现 Batch Normalization（各个网络层输出的归一化）时，主要用到nn.moments和batch_normalization
        其中moments作用是统计矩，mean 是一阶矩，variance 则是二阶中心矩
        tf.nn.moments 计算返回的 mean 和 variance 作为 tf.nn.batch_normalization 参数进一步调用
        :param inputs: 一个有2个或更多维度的张量，第一个维度是batch_size
        :param epsilon: 很小的数值，防止区域划分错误
        :param scope: 
        :return: 返回一个与inputs相同shape和数据的dtype
    """
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.get_variable("beta", params_shape, initializer=tf.zeros_initializer())
        gamma = tf.get_variable("gamma", params_shape, initializer=tf.ones_initializer())
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta
        
    return outputs

### Mask

In [28]:
def mask(inputs, queriers=None, keys=None, type=None):
    """
        对Keys或Queries进行遮盖
    :param inputs: (N, T_q, T_k)
    :param queries: (N, T_q, d)
    :param keys: (N, T_k, d)
    :return:
    
    """
    padding_num = -2 **32 +1
    if type in ("k","key","keys"):
        """
        代码里 if type in ("k", "key", "keys"):  部分是padding mask，
        因为Q乘以V，V的序列后面有很长一部分是全零的向量（这就是我们自定义的padding的对应embedding，我们定义为全0），
        因此全零的部分我们让attention的权重为一个很小的值-4.2949673e+09。
        """
        # 生成 masks
        masks = tf.sign(tf.reduce_sum(tf.abs(keys), axis=-1))  # 维度：（N， T_k），sign 判断出1,0，,1等值
        masks = tf.expand_dims(masks, 1)  # 进行维度的扩展,在下标为1的维度 （N,1,T_k）
        masks = tf.tile(masks, [1, tf.shape(queriers)[1], 1]) # 按照列表的形式，进行维度的扩大制定的倍数(N,T_q,t_k)
        
        # 将 masks 输入到 inputs中去
        paddings = tf.ones_like(inputs) * padding_num
        # tf.where(input, a, b)，返回的是b，b中元素input = true位置由a中对应位置的元素替代，其他位置元素不变
        outputs = tf.where(tf.equal(masks,0),paddings, inputs) #(N,T_q,t_k)
    elif type in ("q","query","queries"):
        """
        似的，query序列最后面也有可能是一堆padding，不过对queries做padding mask不需要把padding加上一个很小的值，
        只要将其置零就行，因为outputs是先key mask，再经过softmax，再进行query mask的。
        """
        # 生成masks
        masks = tf.sign(tf.reduce_sum(tf.abs(queriers),axis=1)) # # (N, T_q)
        masks = tf.expand_dims(masks, -1)  # (N, T_q, 1)
        masks = tf.tile(masks, [1,1, tf.shape(keys)[1]])  # # (N, T_q, T_k)
        
        outputs = inputs * masks
        
    elif type in ("f","future","right"):
        """
        部分则是我们在做decoder的self attention时要用到的sequence mask，
        也就是说在每一步，第i个token关注到的attention只有可能是在第i个单词之前的单词，因为它按理来说，看不到后面的单词。
        """
        diag_vals = tf.ones_like(inputs[0,:,:])  # (T_q, T_k)
        tri1 = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()  # (T_q, T_k)
        masks = tf.expand_dims(tri1,0)
        masks = tf.tile(masks, [tf.shape[0], 1, 1])  # (N, T_q, T_k)
        
        paddings = tf.ones_like(masks) * padding_num
        outputs = tf.where(tf.equal(masks,0),paddings, inputs)
        
    else:
        print("Check if you entered type correctly!")
    
    return outputs
        


###  Context-Attention

也就是论文里提到的Encoder-Decoder Attention，是两个不同序列之间的attention，与来源于自身的 self-attention 相区别。context-attention有很多，这里使用的是scaled dot-product。通过 query 和 key 的相似性程度来确定 value 的权重分布。


实际上这部分代码就是self attention用到的QKV的公式的核心代码，不管是Encoder-Decoder Attention还是Self Attention都是用的这里的scaled dot-product方法。

In [29]:
def scaled_dot_product_attention(Q,K,V,causality = False, dropout_rate=0.,
                                 training=True,scope="scaled_dot_product_attention"):
    """
    查看原论文中3.2.1attention计算公式：Attention(Q,K,V)=softmax(Q K^T /√dk ) V
        :param Q: 查询，三维张量，[N, T_q, d_k].
        :param K: keys值，三维张量，[N, T_k, d_v].
        :param V: values值，三维张量，[N, T_k, d_v].
        :param causality: 布尔值，如果为True，就会对未来的数值进行遮盖
        :param dropout_rate: 0到1之间的一个数值
        :param training: 布尔值，用来控制dropout
        :param scope: 
    """
    with tf.variable_scope(scope,reuse=tf.AUTO_REUSE):
        d_k = Q.get_shape().as_list()[-1]
        
        # dot product
        outputs = tf.matmul(Q, tf.transpose(K, [0,2,1]))   #(N, T_q, T_k)
        
        # scale
        outputs /= d_k ** 0.5
        
        # key masking
        outputs = mask(outputs,Q, K, type="key")
        
        
        # 对未来的数值进行遮盖
        if causality:
            outputs = mask(outputs, type="future")
            
        # softmax 
        outputs = tf.nn.softmax(outputs)
        attention = tf.transpose(outputs, [0,2,1])
        tf.summary.image("attention", tf.expand_dims(attention[:1], -1))
        
        # query masking
        outputs = mask(outputs, Q, K, type="query")
        
        # dropout
        outputs = tf.layers.dropout(outputs, rate=dropout_rate, training=training)
        
        # weighted sum (context vectors), 也是就是输出的最后的 Z
        outputs = tf.matmul(outputs, V)   # (N, T_q, d_v)
        
    return outputs
        

### Multi-head attention

多头self attention就是Transoformer的核心，就是用上面提到的QKV公式算出分布之后，用h份合在一起来表示，论文中的h为8。

这部分代码主要是先产生QKV向量，然后按照h头来进行划分，然后调用上面的scaled dot-product的方法来计算的。

另外这里可以看到代码里将8份self attention分别计算后后concat起来了，然后在self attention层后接了残差连接和layer normalization。

In [30]:
def multihead_attention(queries,keys,values,
                       num_heads = 8,
                       droupout_rate = 0,
                       training =True,
                       causality=False,
                       scope = "multihead_attention"):
    """
        查看原论文中3.2.2中multihead_attention构建，
        这里是将不同的Queries、Keys和values方式线性地投影h次是有益的。
        线性投影分别为dk，dk和dv尺寸。在每个预计版本进行queries、keys、values，
        然后并行执行attention功能，产生dv维输出值。这些被连接并再次投影，产生最终值
        :param queries: 三维张量[N, T_q, d_model]
        :param keys: 三维张量[N, T_k, d_model]
        :param values: 三维张量[N, T_k, d_model]
        :param num_heads: heads数
        :param dropout_rate: 
        :param training: 控制dropout机制
        :param causality: 控制是否遮盖
        :param scope: 
        :return: 三维张量(N, T_q, C) 
    
    """
    d_model = queries.get_shape().as_list()[-1]
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        # 线性 拼接？？ dense ：全连接层  相当于添加一个层 
        # https://blog.csdn.net/yangfengling1023/article/details/81774580
        Q = tf.layers.dense(queries,d_model, use_bias=False) # (N, T_q, d_model)
        K = tf.layers.dense(keys, d_model, use_bias=False) # (N, T_k, d_model)
        V = tf.layers.dense(values,d_model, use_bias=False) #(N, T_k, d_model)
        
        # 再进行切分，切分8个？？,再 拼接？？
#https://blog.csdn.net/SangrealLilith/article/details/80272346?depth_1-utm_source=distribute.pc_relevant.none-task&utm_source=distribute.pc_relevant.none-task
        Q_ = tf.concat(tf.split(Q, num_heads), axis=2)  # (h*N, T_q, d_model/h)
        K_ = tf.concat(tf.split(K, num_heads), axis=2) # (h*N, T_k, d_model/h)
        V_ = tf.concat(tf.split(V, num_heads), axis=2) #  (h*N, T_k, d_model/h)
        
        # Attention 
        outputs = scaled_dot_product_attention(Q_, K_, V_,causality,
                                              droupout_rate, training)
        
        # 重新调整维度 
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0),axis=2)  # (N, T_q, d_model)
        
        # # Residual connection ????  为什么呢
        outputs += queries
        
        # Normalize
        outputs = ln(outputs)
        
    return outputs


# 这里提一句，所有的attention都是用scaled dot-product的方法来计算的，
# 对于self attention来说，Q=K=V，而对于decoder-encoder attention来说，Q=decoder_input，K=V=memory。


        

###  Positional Embedding

就目前而言，Transformer 架构还没有提取序列顺序的信息，
这个信息对于序列而言非常重要，如果缺失了这个信息，可能我们的结果就是：
所有词语都对了，但是无法组成有意义的语句。因此模型对序列中的词语出现的位置进行编码。论文中使用的方法是在偶数位置使用正弦编码，在奇数位置使用余弦编码。


In [31]:
def positional_encoding(inputs,
                       maxlen,
                       masking=True,
                       scope="positional_encoding"):
    """
    ''
        参看论文3.5，由于模型没有循环和卷积，为了让模型知道句子的编号，
        就必须加入某些绝对位置信息，来表示token之间的关系。  
        positional encoding和embedding有相同的维度，这两个能够相加。
        :param inputs: 
        :param maxlen: 
        :param masking: 
        :param scope: 
        :return: 
    """
    E = inputs.get_shape(),as_list()[-1] # static
    N, T = tf.shape(inputs)[0], tf.shape(inputs)[1] # dynamic
    
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        # 位置下标
        position_ind = tf.tile(tf.expand_dims(tf.range(T),0), [N,1])  # (N, T
        # 第一部分的PE函数:sin和cos的参数
        position_enc = np.array([
            [pos / np.power(10000, (i-i%2)/E) for i in range(E)]
            for pos in range(maxlen)
        ])
        
        # 第二部分，将余弦应用于偶数列，将sin应用于奇数。
        position_enc[:,0::2] = np.sin(position_enc[:,0::2])  # dim 2i
        position_enc[:,1::2] = np.cos(position_enc[:,1::2])  # dim 2i +1
        position_enc = tf.convert_to_tensor(position_enc, tf.float32) ## (maxlen, E)
        
        # lookup
        outputs = tf.nn.embedding_lookup(position_enc,position_ind)
        
        # masks
        if masking:
            outputs = tf.where(tf.equal(inputs, 0), inputs, outputs)
            
        
        return tf.to_float(outputs)

### 前向网络

前向网络是两层全连接层接一个残差连接和layer normalization。　　


In [33]:
def ff(inputs, num_units, scope="positionwise_feedforward"):
    """
    inputs: [N, T, C],
    num_units : 两个整数的列表
    
    返回一个：　和输入维度相同的3d　张量
    """
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        # input layer
        outputs = tf.layers.dense(inputs, num_units[0], activation=tf.nn.relu)
        
        # out layer
        outputs = tf.layers.dense(outputs, num_units[1])
        
        
        # Residual connection
        outputs += inputs
        
        # Normalize
        outputs = ln(outputs)
        
    return outputs

### Label Smoothing技术

In [35]:
def label_smoothing(inputs, epsilon = 0.1):
    """
简单来说就是本来ground truth标签是1的，他改到比如说0.9333，
本来是0的，他改到0.0333，这是一个比较经典的平滑技术了。
    
    """
    V = inputs.get_shape().as_list()[-1]  # 获取通道的数量
    return ((1-epsilon) * inputs) + (epsilon / V)
    

### Noam计划衰减学习率

In [36]:

def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    """
    init_lr: 初始学习率
    warmup_steps：　预热步数
    global_step：　全局步数，　从１　开始
    ｌearning Rate = init_lr * warmup_steps**0.5 * min(global_step * warmup_steps**-1.5, warmup_steps**-0.5)
    """
    
    step = tf.cast(global_step + 1, dtype=tf.float32)
    
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)

#  uitls代码

In [37]:
def convert_idx_to_token_tensor(inputs, idx2token):
    """
    将　int32的张量转化成　　字符串张量
    
    inputs: 1 维的 int32 的张量　索引
    idx2token : 字典
    返回：　１维的字符串张量
    """
    
    def my_func(inputs):
        return " ".join(idx2token[elem] for elem in inputs)
    # tf.py_func()接收的是tensor，然后将其转化为numpy array送入我们自定义的my_func函数，
    # 最后再将my_func函数输出的numpy array转化为tensor返回。
    # https://blog.csdn.net/aaon22357/article/details/82996436
    return tf.py_func(my_func, [inputs], tf.string)

###  数据加载方面的代码

### encode函数用于将字符串转化为数字

encode函数用于将字符串转化为数字，这里具体方法是输入的是一个字符序列，然后根据空格切分，然后如果是源语言，则每一句话后面加上"“/s”"，如果是目标语言，则在每一句话前面加上"“s”"，后面加上"“/s”"，然后再转化成数字序列。如果是中文，这里很显然要改，具体看是字符级别输入还是词语级别输入.

In [38]:
def encode(inp, type, dict):
    """
    将　１维的　数组
    
    type: "x" 源语言，　ｙ,目标语言
    dict:   token2idx 字典
    返回：　数字列表
    """
    inp_str = inp.decode("utf-8")
    if type == "x":
        tokens = inp_str.split() + ["</s>"]
    else:
        tokens = ["<s>"] + inp_str.split() + ["</s>"]
        
    x = [dict.get(t, dict["<unk>"]) for t in tokens]
    
    return x


    
    

### generator_fn方法生成训练和评估集数据

这段代码简单讲一下，对于每一个sent1，sent2（源句子，目标句子），sent1经过前面的encode函数转化成x，sent2经过前面的encode函数转化成y之后，decoder的输入decoder_input是y[:-1]，预期输出y是y[1:]，啥意思呢，就是其实是RNN一样的，用来解码输入的前N-1个，期望的输出是从第2个到第N个，也是N-1个。

In [39]:
def generator_fn(sents1, sents2, vocab_fpath):
    """
     sents1: 源句子列表
    sents2: 目标句子列表
    vocab_fpath: s

    yields
    xs: tuple of
        x: list of source token ids in a sent
        x_seqlen: int. sequence length of x
        sent1: str. raw source (=input) sentence
    labels: tuple of
        decoder_input: decoder_input: list of encoded decoder inputs
        y: list of target token ids in a sent
        y_seqlen: int. sequence length of y
        sent2: str. target sentence
    """
    token2idx, _ = load_vocab(vocab_fpath)
    for sent1, sent2 in zip(sents1, sents2):
        x = encode(sent1, "x", token2idx)
        y = encode(sent2, "y", token2idx)
        decoder_input, y = y[:-1], y[1:]
        
        x_seqlen, y_seqlen = len(x), len(y)
        
        yield (x, x_seqlen, sent1), (decoder_input, y, y_seqlen, sent2)
        

### input_fn方法用来生成Batch数据

这段代码其实也比较值得学习，用tf.data.Dataset.from_generator的方式读入数据，不受计算图的影响，比较好。Dataset作为新的API，比以前的feed_dict效率要高一些。关于dataset的简单使用，和一些它代码里用到的API的简单解释


In [40]:
def input_fn(sents1, sents2, vocab_fpath, bath_size, shuffle=False):
    """
    成批投放数据
    sents1: 源句子列表
    sents2: 目标句子列表
    vocab_fpath: string. vocabulary file path.
    batch_size: scalar 每个批次的大小　张量
    shuffle: boolean

    Returns
    xs: tuple of
        x: int32 tensor. (N, T1)
        x_seqlens: int32 tensor. (N,)
        sents1: str tensor. (N,)
    ys: tuple of
        decoder_input: int32 tensor. (N, T2)
        y: int32 tensor. (N, T2)
        y_seqlen: int32 tensor. (N, )
        sents2: str tensor. (N,)
    """
    shape = (([None],(),()),
            ([None], [None], (),()))
    types = ((tf.int32, tf.int32, tf.string),
            (tf.int32, tf.int32, tf.int32, tf.string))
    
    paddings = ((0,0,''),
               (0,0,0,''))
    
    dataset = tf.data.Dataset.from_generator(
            generator_fn,output_shapes=shape,
            output_types=types,
            args=(sents1, sents2, vocab_fpath)) # <- arguments for generator_fn. converted to np string arrays
    if shuffle:  # for training
        dataset = dataset.shuffle(128 * bath_size)
        
    dataset = dataset.repeat()  # 一直迭代下去
    dataset = dataset.padded_batch(bath_size, shapes, paddings).prefetch(1)
    
    return dataset

# 整合模型

In [51]:
class Transformer:
    """
    xs: tuple of
        x: int32 tensor. (N, T1)
        x_seqlens: int32 tensor. (N,)
        sents1: str tensor. (N,)
    ys: tuple of
        decoder_input: int32 tensor. (N, T2)
        y: int32 tensor. (N, T2)
        y_seqlen: int32 tensor. (N, )
        sents2: str tensor. (N,)
    training: boolean
    
    """
    
    def __init__(self, hp):  # hp 输入向量
        self.hp = hp
        self.token2idx, self.idx2token = load_vocab(hp.vocab)
        self.embeddings = get_token_embeddings(self.hp.vocab_size, self.hp.d_model,zero_pad=True)
        
    def encode(self, xs, training=True):   # xs 是　input_fn方法用来生成Batch数据　返回的数值
        """
        返回：memory: encoder outputs. (N, T1, d_model)
        """
        with tf.variable_scope("encoder", reuse=tf.AUTO_REUSE):
            x, seqlens, sents1 = xs
                
            # emedding 
            enc = tf.nn.embedding_lookup(self.embeddings,x) #(N, T1, d_model)
            enc *= self.hp.d_model ** 0.5  # scale
            
            enc += positional_encoding(enc, self.hp.maxlen1)
            enc = tf.layers.dropout(enc, self.hp.dropout_rate, training=training)
            
            # Blocks
            for i in range(self.hp.num_blocks):
                with tf.variable_scope("num_blocaks_{}".format(i), reuse=tf.AUTO_REUSE):
                    # self-attention
                    enc = multihead_attention(queries=enc,
                                             keys = enc,
                                             values =enc,
                                             num_heads=self.hp.num_heads,
                                             droupout_rate=self.hp.dropout_rate,
                                             training=training,
                                             causality=False)
                    # feed forward
                    enc = ff(enc, num_units=[self.hp.d_ff, self.hp.d_model])
                    
            memory = enc
            return memory, sents1
        
    def decode(self, ys, memory, training=True):
        """
        memory: encoder outputs, (N, T1, d_model)

        Returns
        logits: (N, T2, V). float32.
        y_hat: (N, T2). int32
        y: (N, T2). int32
        sents2: (N,). string.
        """
        with tf.variable_scope("decoder",reuse=tf.AUTO_REUSE):
            decode_inputs, y, seqlens, sents2 = ys   #  也是　input_fn方法用来生成Batch数据　返回的数值

            # embedding 
            dec = tf.nn.embedding_lookup(self.embeddings, decode_inputs)  # (N, T2, d_model)
            dec *= self.hp.d_model ** 0.5 # 标量
            dec += positional_encoding(dec, self.hp.maxlen2)
            dec = tf.layers.dropout(dec, self.hp.dropout_rate, training=training)

            # Blocks
            for i in range(self.hp.num_blocks):
                with tf.variable_scope("num_blocks_{}".format(i), reuse=tf.AUTO_REUSE):
                 # mask self-attention (请注意，因果关系此时为True)
                    dec = multihead_attention(queries=dec,
                                             keys=dec,
                                             values=dec,
                                             num_heads=self.hp.num_heads,
                                             droupout_rate=self.hp.dropout_rate,
                                             training=training,
                                             causality=True,
                                             scope="self_attention")

                # 普通的attention
                    dec = multihead_attention(queries=dec,
                         keys=dec,
                         values=dec,
                         num_heads=self.hp.num_heads,
                         droupout_rate=self.hp.dropout_rate,
                         training=training,
                         causality=False,
                         scope="vanilla_attention")

                    ## feed Forward
                    dec = ff(dec, num_units=[self.hp.d_ff, self.hp.d_model])

        # 最后的线性连接层，　embedding 权重是共享的
        weights = tf.transpose(self.embeddings) # (d_model, vocab_size)
# https://blog.csdn.net/weixin_39274659/article/details/87869527?depth_1-utm_source=distribute.pc_relevant.none-task&utm_source=distribute.pc_relevant.none-task
       #  tf.einsum 的详细使用方法介绍 
        logits = tf.einsum("ntd,dk->ntk",dec, weights)  #(N, T2, vocab_size)
        y_hat = tf.to_int32(tf.argmax(logits, axis=-1))

        return logits, y_hat, y,sents2
        
        
        
    def train(self, xs, ys):
        """
        Returns
        loss: scalar.
        train_op: training operation
        global_step: scalar.
        summaries: training summary node
        
        """
        # forward
        memory, sents1 = self.encode(xs)
        
        logits, preds, y, sents2 = self.decode(ys, memory)
        
        # train scheme
        y_ = label_smoothing(tf.one_hot(y, depth=self.hp.vocab_size))
        ce = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_)
        nonpadding = tf.to_float(tf.not_equal(y,self.token2idx["<pad>"]))  # 0: <pad>
        #  测试一下********************************************
        print(tf.reduce_sum(nonpadding))
        # ***************************************************
        loss = tf.reduce_sum(ce * nonpadding) / (tf.reduce_sum(nonpadding) + 1e-7)
        
        global_step = tf.train.get_or_create_global_step()
        # 这里用了一个Noam计划衰减学习率
        lr = noam_scheme(self.hp.lr, global_step, self.hp.warmup_steps)
        optimizer = tf.train.AdamOptimizer(lr)
        train_op = optimizer.minimize(loss, global_step=global_step)
        
        
        tf.summary.scalar("lr",lr)
        tf.summary.scalar("loss",loss)
        tf.summary.scalar("global_step",global_step)
        
        
        summaries = tf.summary.merge_all()
        
        return loss, train_op, global_step, summaries
    
    def enval(self, xs, ys):
        """
        自我预测
        忽略输入y
        Returns
        y_hat: (N, T2)
        """
        decoder_inputs, y, y_seqlen, sents = ys
        
        decoder_inputs = tf.ones((tf.shape(xs[0])[0],1),tf.int32) * self.token2idx["<s>"]
        ys = (decode, y, y_seqlen, sents2)
        
        memory, sents1 = self.encode(xs, False)
        logging.info("Inference graph is being built. Please be patient.")
        for _ in tqdm(range(self.hp.maxlen2)):
            logits, y_hat, y, sents2 = self.decode(ys, memory, False)
            if tf.reduce_sum(y_hat,1) == self.token2idx["<pad>"]:
                break
            
            _decoder_inputs = tf.concat((decoder_inputs, y_hat),1)
            ys = (_decoder_inputs, y, y_seqlen, sents2)
            
        #监测随机样本
        n = tf.random_uniform((), 0, tf.shape(y_hat)[0]-1, tf.int32)
        sent1 = sents1[n]
        pred = convert_idx_to_token_tensor(y_hat[n], self.idx2token)
        sent2 =sents2[n]
        
        tf.summary.text("sent1",sent1)
        tf.summary.text("pred",pred)
        rf.summary.text("sent2",sent2)
        summaries = tf.summary.merge_all()
        
        return y_hat, summaries
        